# f_neuralNetwork
----

Written in the Python 3.7.9 Environment with the following package versions

    * joblib 1.0.1
    * numpy 1.19.5
    * pandas 1.3.1
    * scikit-learn 0.24.2
    * tensorflow 2.5.0

By Nicole Lund 

This Jupyter Notebook tunes a neural network model for Exoplanet classification from Kepler Exoplanet study data.

Column descriptions can be found at https://exoplanetarchive.ipac.caltech.edu/docs/API_kepcandidate_columns.html 

**Source Data**

The source data used was provided by University of Arizona's Data Analytics homework assignment. Their data was derived from https://www.kaggle.com/nasa/kepler-exoplanet-search-results?select=cumulative.csv

The full data set was released by NASA at
https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=koi

In [9]:
# Import Dependencies

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt

# Data manipulation
import numpy as np
import pandas as pd
from statistics import mean
from operator import itemgetter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical


# Parameter Selection
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV

# Model Development
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import load_model

# Model Metrics
from sklearn.metrics import classification_report

# Save files
import joblib



# # Ignore deprecation warnings
# import warnings
# warnings.simplefilter('ignore', FutureWarning)

In [10]:
# Set the seed value for the notebook, so the results are reproducible
from numpy.random import seed
seed(1)

# Read the CSV and Perform Basic Data Cleaning

In [11]:
# Import data
df = pd.read_csv("b_source_data/exoplanet_data.csv")
# print(df.info())

# Drop columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop rows containing null values
df = df.dropna()

# Display data info
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6991 entries, 0 to 6990
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   koi_disposition    6991 non-null   object 
 1   koi_fpflag_nt      6991 non-null   int64  
 2   koi_fpflag_ss      6991 non-null   int64  
 3   koi_fpflag_co      6991 non-null   int64  
 4   koi_fpflag_ec      6991 non-null   int64  
 5   koi_period         6991 non-null   float64
 6   koi_period_err1    6991 non-null   float64
 7   koi_period_err2    6991 non-null   float64
 8   koi_time0bk        6991 non-null   float64
 9   koi_time0bk_err1   6991 non-null   float64
 10  koi_time0bk_err2   6991 non-null   float64
 11  koi_impact         6991 non-null   float64
 12  koi_impact_err1    6991 non-null   float64
 13  koi_impact_err2    6991 non-null   float64
 14  koi_duration       6991 non-null   float64
 15  koi_duration_err1  6991 non-null   float64
 16  koi_duration_err2  6991 

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

These features were chosen in a_tree_feature_selection.ipynb and b_forest_feature_selections.ipynb

In [12]:
# Set features from Decision Tree and Random Forest assessment.
tree_features = ['koi_fpflag_nt', 'koi_fpflag_co', 'koi_fpflag_ss', 'koi_model_snr']
forest_features = ['koi_fpflag_co', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_model_snr', 'koi_prad']
tree_forest_selected_features = set(tree_features + forest_features)
tree_forest_selected_features

{'koi_fpflag_co',
 'koi_fpflag_nt',
 'koi_fpflag_ss',
 'koi_model_snr',
 'koi_prad'}

In [13]:
# Choose all features that are not associated with error measurements
# print(df.columns)
non_error_selected_features = ['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration','koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_slogg', 'koi_srad', 'ra', 'dec', 'koi_kepmag']
non_error_selected_features

['koi_fpflag_nt',
 'koi_fpflag_ss',
 'koi_fpflag_co',
 'koi_fpflag_ec',
 'koi_period',
 'koi_time0bk',
 'koi_impact',
 'koi_duration',
 'koi_depth',
 'koi_prad',
 'koi_teq',
 'koi_insol',
 'koi_model_snr',
 'koi_tce_plnt_num',
 'koi_steff',
 'koi_slogg',
 'koi_srad',
 'ra',
 'dec',
 'koi_kepmag']

In [14]:
# Split dataframe into X and y
X = df.drop("koi_disposition", axis=1)
# X = df[non_error_selected_features]
# X = df[tree_forest_selected_features]
y = df["koi_disposition"]
print(X.shape, y.shape)

(6991, 40) (6991,)


# Create a Train Test Split

Use `koi_disposition` for the y values

In [15]:
# Split X and y into training and testing groups
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [16]:
# Display training data
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
4954,0,0,0,0,97.942437,0.001183,-0.001183,157.97325,0.00888,-0.00888,...,-198,3.806,0.445,-0.194,2.284,0.726,-1.090,288.53769,51.280384,11.366
4235,0,0,1,1,16.609490,0.000141,-0.000141,147.61311,0.00715,-0.00715,...,-210,4.395,0.105,-0.195,1.050,0.323,-0.162,300.43979,43.930729,14.926
848,0,0,0,0,32.275385,0.000541,-0.000541,149.04600,0.01400,-0.01400,...,-103,4.123,0.182,-0.098,1.483,0.230,-0.316,298.29495,48.775459,14.167
2874,0,1,1,1,0.787120,0.000002,-0.000002,131.73939,0.00190,-0.00190,...,-201,4.481,0.150,-0.100,0.787,0.093,-0.103,295.28110,40.132858,14.964
3016,0,0,0,0,6.963527,0.000039,-0.000039,133.01717,0.00435,-0.00435,...,-201,3.978,0.458,-0.153,1.686,0.413,-0.709,296.10406,42.342590,13.128


# Pre-processing

In [17]:
# Scale the data with MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# One-Hot-Encode the y data

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [19]:
y_train_categorical.shape[1]

3

In [20]:
print('Unique KOI Disposition Values')
print(y.unique())
print('-----------')
print('Sample KOI Disposition Values and Encoding')
print(y_test[:5])
print(y_test_categorical[:5])

Unique KOI Disposition Values
['CONFIRMED' 'FALSE POSITIVE' 'CANDIDATE']
-----------
Sample KOI Disposition Values and Encoding
4982    FALSE POSITIVE
4866         CANDIDATE
2934    FALSE POSITIVE
5007    FALSE POSITIVE
3869    FALSE POSITIVE
Name: koi_disposition, dtype: object
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


# Create and Train the Model - Neural Network



In [21]:
# Create model
model = Sequential()

In [22]:
# Define neural network parameters
number_inputs = X_train_scaled.shape[1]
number_outputs = y_train_categorical.shape[1]
number_hidden_nodes = int(mean([number_inputs, number_outputs]))+1
number_hidden_nodes

22

In [23]:
# Define first layer
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

# Define output layer
model.add(Dense(units=number_outputs, activation='softmax'))                

In [24]:
# Review Model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 22)                902       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 69        
Total params: 971
Trainable params: 971
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
# Train model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
153/153 - 0s - loss: 0.9104 - accuracy: 0.5248
Epoch 2/100
153/153 - 0s - loss: 0.6507 - accuracy: 0.7427
Epoch 3/100
153/153 - 0s - loss: 0.4845 - accuracy: 0.8048
Epoch 4/100
153/153 - 0s - loss: 0.4196 - accuracy: 0.8183
Epoch 5/100
153/153 - 0s - loss: 0.3907 - accuracy: 0.8189
Epoch 6/100
153/153 - 0s - loss: 0.3755 - accuracy: 0.8232
Epoch 7/100
153/153 - 0s - loss: 0.3654 - accuracy: 0.8316
Epoch 8/100
153/153 - 0s - loss: 0.3564 - accuracy: 0.8298
Epoch 9/100
153/153 - 0s - loss: 0.3501 - accuracy: 0.8308
Epoch 10/100
153/153 - 0s - loss: 0.3442 - accuracy: 0.8402
Epoch 11/100
153/153 - 0s - loss: 0.3401 - accuracy: 0.8392
Epoch 12/100
153/153 - 0s - loss: 0.3363 - accuracy: 0.8394
Epoch 13/100
153/153 - 0s - loss: 0.3321 - accuracy: 0.8439
Epoch 14/100
153/153 - 0s - loss: 0.3275 - accuracy: 0.8471
Epoch 15/100
153/153 - 0s - loss: 0.3240 - accuracy: 0.8531
Epoch 16/100
153/153 - 0s - loss: 0.3216 - accuracy: 0.8539
Epoch 17/100
153/153 - 0s - loss: 0.3210 - accura

In [27]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

66/66 - 0s - loss: 0.2895 - accuracy: 0.8785
Loss: 0.2894859313964844, Accuracy: 0.8784556984901428


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [45]:
# Code was modified from sample code presented on
# https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
# 
# Function to create model, required for KerasClassifier
def create_model(neurons=40):
	# create model
	model = Sequential()
	model.add(Dense(neurons, input_dim=number_inputs, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(units=number_outputs, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [48]:
# Code was modified from sample code presented on
# https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

# Use scikit-learn to grid search the batch size and epochs

# create model
model = KerasClassifier(build_fn=create_model, verbose=3)

# define the grid search parameters
batch_size = [10, 20, 30]
epochs = [100, 500, 1000]
neurons = [5, 10, 15, 20, 25, 30, 35, 40]
param_grid = dict(batch_size=batch_size, epochs=epochs, neurons=neurons)

# Apply GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_scaled, y_train_categorical)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.891478 using {'batch_size': 20, 'epochs': 400, 'neurons': 20}
0.876558 (0.010706) with: {'batch_size': 10, 'epochs': 100, 'neurons': 20}
0.885551 (0.005627) with: {'batch_size': 10, 'epochs': 100, 'neurons': 30}
0.889230 (0.011441) with: {'batch_size': 10, 'epochs': 100, 'neurons': 40}
0.888821 (0.009555) with: {'batch_size': 10, 'epochs': 200, 'neurons': 20}
0.881463 (0.008515) with: {'batch_size': 10, 'epochs': 200, 'neurons': 30}
0.884733 (0.007476) with: {'batch_size': 10, 'epochs': 200, 'neurons': 40}
0.880850 (0.013399) with: {'batch_size': 10, 'epochs': 400, 'neurons': 20}
0.888412 (0.008554) with: {'batch_size': 10, 'epochs': 400, 'neurons': 30}
0.888821 (0.007825) with: {'batch_size': 10, 'epochs': 400, 'neurons': 40}
0.878602 (0.006253) with: {'batch_size': 20, 'epochs': 100, 'neurons': 20}
0.880850 (0.006907) with: {'batch_size': 20, 'epochs': 100, 'neurons': 30}
0.884938 (0.006259) with: {'batch_size': 20, 'epochs': 100, 'neurons': 40}
0.888412 (0.005773) with: {'ba

In [ ]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)

In [ ]:
print(classification_report(y_test, predictions,
                            target_names=y.unique()))

# Model Results when using all features
* X Definition: X = df.drop("koi_disposition", axis=1)
* 

# Model Results when using all features not associated with error measurements
* X Definition: X = df[non_error_selected_features]
* 

# Model Results when using selected features from Decision Tree and Random Forest Classifiers
* X = df[tree_forest_selected_features]
* 